<a href="https://colab.research.google.com/github/serzhoon/NoSQL_NCFU/blob/main/lab06_NoSQL_DynamoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Отчёт по лабораторной работе на тему: DynamoDB

### Выполнил студент группы ПИН-Б-З-22-1

### Гадиян Сергей Гариевич

###Задачи

1. Используя найденную вами формулу для расчета числа разделов, используемых для таблицы,
рассчитайте, сколько разделов будет использовано для таблицы объемом 100 ГБ данных,
назначенной 2000 RCUs и 3000 WCUs.


Количество разделов в таблице DynamoDB рассчитывается на основе пропускной способности (RCUs и WCUs) и объёма хранимых данных. По умолчанию DynamoDB автоматически разделяет данные и увеличивает количество разделов по мере роста таблицы.
Формула для начального расчёта:
$(readCapacityUnits / 3000) + (writeCapacityUnits / 1000) = initialPartitions (округлённое вверх)$

Деление целочисленное, с округлением вверх (ceil).<br>
Шаги:<br>
Объем данных 100 ГБ / 10 = 10. Округляем вверх 10.<br>
RCU: 2000 / 3000 = 0.6.... Округляем вверх 1.<br>
WCU: 3000 / 1000 = 3 Округляем вверх 3.<br>
Берем максимальное из этих трех чисел: MAX(10, 1, 3) = 10.

In [ ]:
import math
gb = float(input("Размер данных в ГБ: "))
rcu = float(input("RCU: "))
wcu = float(input("WCU: "))
p1 = math.ceil(gb / 10)
p2 = math.ceil(rcu / 3000)
p3 = math.ceil(wcu / 1000)

print(f"Нужно разделов: {max(p1, p2, p3)}")

Размер данных в ГБ: 100
RCU: 2000
WCU: 3000
Нужно разделов: 10


2. Если бы вы хранили твиты в DynamoDB, как бы вы это делали, используя поддерживаемые типы
данных DynamoDB?

Я бы хранил твиты в DynamoDB так:
*  Основная таблица Tweets с ключом (user_id, tweet_id)
*  Отдельная таблица TweetHashtags для быстрого поиска по хештегам
*  GSI TimestampIndex для сортировки твитов по времени

Использовал бы все типы DynamoDB:

*  S - для текста, ID, дат
*  N - для счётчиков лайков/ретвитов
*  SS - для хештегов и упоминаний
*  BOOL - для флагов (верификация, ретвит)
*  L и M - для вложенных объектов (медиа, геолокация)

3. Помимо операций PutItem, DynamoDB также предлагает операции обновления элементов,
которые позволяют изменять элемент, если условное выражение выполнено. Ознакомьтесь с
документацией по условным выражениям и выполните условную операцию обновления
элемента в таблице ShoppingCart

In [ ]:
# я создал таблицу
dynamo create-table \
  --table-name ShoppingCart \
  --attribute-definitions AttributeName=ItemName,AttributeType=S \
  --key-schema AttributeName=ItemName,KeyType=HASH \
  --provisioned-throughput ReadCapacityUnits=1,WriteCapacityUnits=1

# добавил товары с количеством и ценой
dynamo put-item \
  --table-name ShoppingCart \
  --item '{
    "ItemName": {"S": "iPhone 15"},
    "Quantity": {"N": "2"},
    "Price": {"N": "999.99"},
    "Category": {"S": "Electronics"},
    "InStock": {"BOOL": true}
  }'

dynamo put-item \
  --table-name ShoppingCart \
  --item '{
    "ItemName": {"S": "Python Book"},
    "Quantity": {"N": "10"},
    "Price": {"N": "39.99"},
    "Category": {"S": "Books"},
    "InStock": {"BOOL": true}
  }'

dynamo put-item \
  --table-name ShoppingCart \
  --item '{
    "ItemName": {"S": "Tickle Me Elmo"},
    "Quantity": {"N": "0"},
    "Price": {"N": "29.99"},
    "Category": {"S": "Toys"},
    "InStock": {"BOOL": false}
  }'

In [ ]:
# промежуточный вид текущей таблицы
sergey@sergey-VMware-Virtual-Platform:~$ dynamo scan --table-name ShoppingCart
{
    "Items": [
        {
            "ItemName": {
                "S": "Tickle Me Elmo"
            },
            "Quantity": {
                "N": "0"
            },
            "Category": {
                "S": "Toys"
            },
            "Price": {
                "N": "29.99"
            },
            "InStock": {
                "BOOL": false
            }
        },
        {
            "ItemName": {
                "S": "iPhone 15"
            },
            "Quantity": {
                "N": "2"
            },
            "Category": {
                "S": "Electronics"
            },
            "Price": {
                "N": "999.99"
            },
            "InStock": {
                "BOOL": true
            }
        },
        {
            "ItemName": {
                "S": "Python Book"
            },
            "Quantity": {
                "N": "10"
            },
            "Category": {
                "S": "Books"
            },
            "Price": {
                "N": "39.99"
            },
            "InStock": {
                "BOOL": true
            }
        }
    ],
    "Count": 3,
    "ScannedCount": 3,
    "ConsumedCapacity": null
}
(END)


In [ ]:
# увеличим количество iPhone только если он в наличии
sergey@sergey-VMware-Virtual-Platform:~$ dynamo update-item \
  --table-name ShoppingCart \
  --key '{"ItemName": {"S": "iPhone 15"}}' \
  --update-expression "SET Quantity = Quantity + :inc" \
  --condition-expression "InStock = :true" \
  --expression-attribute-values '{
    ":inc": {"N": "1"},
    ":true": {"BOOL": true}
  }'

In [ ]:
# проверка
sergey@sergey-VMware-Virtual-Platform:~$ dynamo get-item \
  --table-name ShoppingCart \
  --key '{"ItemName": {"S": "iPhone 15"}}'
{
    "Item": {
        "ItemName": {
            "S": "iPhone 15"
        },
        "Quantity": {
            "N": "3" # увеличилось на 1
        },
        "Category": {
            "S": "Electronics"
        },
        "InStock": {
            "BOOL": true
        },
        "Price": {
            "N": "999.99"
        }
    }
}